Recurrent Neural Networks

In [2]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

17464789/17464789 [==============================] - 0s 0us/step


In [3]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

In [ ]:
results = model.evaluate(test_data, test_labels)
print(results)

In [ ]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)

In [ ]:
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]

In [10]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

1/1 [==============================] - 1s 599ms/step
[0.7433617]
1/1 [==============================] - 0s 21ms/step
[0.21627526]


Recurrent Neural Network Play Generator

In [11]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [14]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

Saving bee-movie-script.txt to bee-movie-script.txt


In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

In [18]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

In [21]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [22]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [23]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))

In [46]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

In [48]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 70) # (batch_size, sequence_length, vocab_size)


In [49]:
pred = example_batch_predictions[0]

sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars

'Bw BQnt1HWeohi4P UBjN dSborOR4-?!fr5A "BDL"jUM 3Jkai.v96WtD24EUwf:cF8a2NMifzo\rF7GPULFLT!Jx76b,3b,Gy\r'

In [50]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [51]:
model.compile(optimizer='adam', loss=loss)

In [52]:
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [53]:
history = model.fit(data, epochs=200, callbacks=[checkpoint_callback])

Epoch 1/200
8/8 [==============================] - 3s 193ms/step - loss: 3.7593
Epoch 2/200
8/8 [==============================] - 1s 153ms/step - loss: 3.2851
Epoch 3/200
8/8 [==============================] - 1s 111ms/step - loss: 3.2140
Epoch 4/200
8/8 [==============================] - 1s 160ms/step - loss: 3.1532
Epoch 5/200
8/8 [==============================] - 1s 94ms/step - loss: 3.0319
Epoch 6/200
8/8 [==============================] - 1s 89ms/step - loss: 2.7961
Epoch 7/200
8/8 [==============================] - 1s 89ms/step - loss: 2.5614
Epoch 8/200
8/8 [==============================] - 1s 92ms/step - loss: 2.4173
Epoch 9/200
8/8 [==============================] - 1s 92ms/step - loss: 2.3245
Epoch 10/200
8/8 [==============================] - 1s 97ms/step - loss: 2.2672
Epoch 11/200
8/8 [==============================] - 1s 104ms/step - loss: 2.2260
Epoch 12/200
8/8 [==============================] - 1s 106ms/step - loss: 2.1877
Epoch 13/200
8/8 [=========================

In [54]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [55]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [56]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension

      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [57]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: bee
bees don't listen!
 I'm notjousingan amusement park into our day.
 That's why we don't need vacations.
 Boy, quite a bit of pomp..
 - Like what? Gixce." Now youcan pick out yourjob and be normal.
 - Well...
 - Well?
 Well, I met someone.
 You, did, crumb.
 - It's fant.
 Would you like some honey with that?
 It is bee-approved. Don't forget these.
 Milk, cream, cheer herp? Anl It's an incredible scenehere in downtwor.
 Make it!
 There's heating, cooling, livitious, bee.
 Sting we wind al the Thiskion could the nice to Ital.
 - Say Bun legitimate businesses.
 I move for a complete is vild smoll no really ste.
 What wis onthis creep, and we can all go home?!
 - Order in this court!
 - You're all thinking it!
 Oorlenomestion Oally? Fiect.
 Ooh, black and yellow!
 We why doe
